In [13]:
%load_ext sparksql_magic

from pyspark.sql import SparkSession

# Start Spark session
spark = (
    SparkSession.builder.appName("FilterGHJson")
    .config("spark.sql.catalogImplementation", "hive")
    .enableHiveSupport()
    .getOrCreate()
)

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [3]:
df = spark.read.json("../data/*.json.gz")

25/02/20 16:59:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df.printSchema()
df.show()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|             type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|{https://avatars....|2025-01-01T01:00:00Z|45186095831|                NULL|{created, NULL, {...|  true|{810155444, aws-a...|IssueCommentEvent|
|{https://avatars....|2025-01-01T01:00:00Z|45186095832|                NULL|{NULL, 1c0bb7ae07...|  true|{314538798, Malee...|        PushEvent|
|{https://avatars....|2025-01-01T01:00:00Z|45186095834|                NULL|{NULL, c2e22ed422...|  true|{463061974, phanu...|        PushEvent|
|{https://avatars....|2025-01-01T01:00:00Z|45186095837|                NULL|{NULL, 40f81c7a80...|  true|{894756355, plume...|        Pus

In [5]:
df = df.filter(df["type"].isin("IssuesEvent", "IssueCommentEvent"))

In [6]:
df.createOrReplaceTempView("df")

In [7]:
df.show(20)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|             type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+-----------------+
|{https://avatars....|2025-01-01T01:00:00Z|45186095831|                NULL|{created, NULL, {...|  true|{810155444, aws-a...|IssueCommentEvent|
|{https://avatars....|2025-01-01T01:00:00Z|45186095924|                NULL|{created, NULL, {...|  true|{841034127, hisRo...|IssueCommentEvent|
|{https://avatars....|2025-01-01T01:00:02Z|45186096107|                NULL|{opened, NULL, NU...|  true|{645980442, mo9a7...|      IssuesEvent|
|{https://avatars....|2025-01-01T01:00:02Z|45186096114|                NULL|{created, NULL, {...|  true|{773201949, s-hir...|IssueCommen

In [11]:
spark.sql(
    "SELECT actor.*,created_at,id,org.*,payload.*,public,repo.*,type FROM df WHERE type = 'IssuesEvent'"
).show(20)

+--------------------+--------------------+-----------+---------+--------------------+--------------------+--------------------+-----------+--------------------+-----------+---------+-------------------+--------------------+--------+------+-------+-------+-----------+-------------+------+----+--------------------+-------------+------+------+-----+------------+-------+-----------+----+--------+-------+-------------+------+----+------+---------+--------------------+--------------------+-----------+
|          avatar_url|       display_login|gravatar_id|       id|               login|                 url|          created_at|         id|          avatar_url|gravatar_id|       id|              login|                 url|  action|before|comment|commits|description|distinct_size|forkee|head|               issue|master_branch|member|number|pages|pull_request|push_id|pusher_type| ref|ref_type|release|repository_id|review|size|public|       id|                name|                 url|       ty

In [9]:
spark.sql(
    """
          SELECT payload.issue.* FROM df
          where type = 'IssuesEvent'
          """
).show(20)

+------------------+--------------------+--------------------+------------------+-----------------------------+--------------------+--------+--------------------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------------+------+---------+------------------+------+------------------------+------------+--------------------+--------------------+------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|active_lock_reason|            assignee|           assignees|author_association|                         body|           closed_at|comments|        comments_url|          created_at|draft|          events_url|            html_url|        id|              labels|          labels_url|locked|milestone|           node_id|number|performed_via_github_app|pull_request|           reactions|      repository_url| state|state_reason|sub_issues_summary|  

In [66]:
spark.sql(
    """
          SELECT payload.* FROM df
          where type = 'IssueCommentEvent' or type = 'IssuesEvent'
          """
).show(20)

+-------+------+--------------------+-------+-----------+-------------+------+----+--------------------+-------------+------+------+-----+------------+-------+-----------+----+--------+-------+-------------+------+----+
| action|before|             comment|commits|description|distinct_size|forkee|head|               issue|master_branch|member|number|pages|pull_request|push_id|pusher_type| ref|ref_type|release|repository_id|review|size|
+-------+------+--------------------+-------+-----------+-------------+------+----+--------------------+-------------+------+------+-----+------------+-------+-----------+----+--------+-------+-------------+------+----+
|created|  NULL|{NULL, NONE, This...|   NULL|       NULL|         NULL|  NULL|NULL|{NULL, NULL, [], ...|         NULL|  NULL|  NULL| NULL|        NULL|   NULL|       NULL|NULL|    NULL|   NULL|         NULL|  NULL|NULL|
|created|  NULL|{NULL, NONE, vers...|   NULL|       NULL|         NULL|  NULL|NULL|{NULL, NULL, [], ...|         NULL|  

In [ ]:
spark.sql(
    """
          SELECT payload.issue.* FROM df
          where type = 'IssueCommentEvent'
          """
).show(20, True)

+------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------+--------------------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------------+------+---------+--------------------+------+------------------------+--------------------+--------------------+--------------------+------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|active_lock_reason|            assignee|           assignees|author_association|                body|           closed_at|comments|        comments_url|          created_at|draft|          events_url|            html_url|        id|              labels|          labels_url|locked|milestone|             node_id|number|performed_via_github_app|        pull_request|           reactions|      repository_url| state|state_reason|sub_issues_summary|

In [117]:
spark.sql(
    """
          SELECT payload.issue.user.* FROM df
          where type = 'IssueCommentEvent'
          """
).show(20, True)

+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+---------+---------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----+--------------------+--------------+
|          avatar_url|          events_url|       followers_url|       following_url|           gists_url|gravatar_id|            html_url|       id|          login|             node_id|   organizations_url| received_events_url|           repos_url|site_admin|         starred_url|   subscriptions_url|type|                 url|user_view_type|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+---------+---------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----+-----

In [17]:
%%sparksql
SELECT 
    created_at, 
    payload.issue.author_association, 
    payload.action, 
    payload.issue.closed_at, 
    payload.issue.title, 
    payload.issue.id, 
    payload.issue.labels, 
    payload.issue.state, 
    payload.issue.state_reason
FROM df 
WHERE (payload.action = 'opened' OR payload.action = 'reopened') 
    AND type = 'IssuesEvent' AND payload.issue.closed_at IS NOT NULL

only showing top 20 row(s)


created_at,author_association,action,closed_at,title,id,labels,state,state_reason
2025-01-01T01:05:06Z,OWNER,opened,2025-01-01T01:05:04Z,🚨 Error Report - 2025. 01. 01. 01:05:03,2764760625,"[Row(color='ededed', default=False, description=None, id=7568837120, name='health', node_id='LA_kwDOGKB-P88AAAABwyNKAA', url='https://api.github.com/repos/RWB0104/RWB0104.github.io/labels/health'), Row(color='ededed', default=False, description=None, id=7568837121, name='prod', node_id='LA_kwDOGKB-P88AAAABwyNKAQ', url='https://api.github.com/repos/RWB0104/RWB0104.github.io/labels/prod')]",closed,completed
2025-01-01T01:25:41Z,NONE,opened,2025-01-01T01:25:40Z,20250101,2764768270,[],closed,completed
2025-01-01T12:23:25Z,OWNER,opened,2025-01-01T12:23:24Z,🚨 Error Report - 2025. 01. 01. 12:23:23,2765039832,"[Row(color='ededed', default=False, description=None, id=7568837120, name='health', node_id='LA_kwDOGKB-P88AAAABwyNKAA', url='https://api.github.com/repos/RWB0104/RWB0104.github.io/labels/health'), Row(color='ededed', default=False, description=None, id=7568837121, name='prod', node_id='LA_kwDOGKB-P88AAAABwyNKAQ', url='https://api.github.com/repos/RWB0104/RWB0104.github.io/labels/prod')]",closed,completed
2025-01-01T12:55:50Z,OWNER,opened,2025-01-01T12:55:49Z,[FE] 로그인,2765056166,"[Row(color='00a800', default=False, description='기능 개발', id=7950417093, name='✨feat', node_id='LA_kwDONSoEw88AAAAB2eG8xQ', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%E2%9C%A8feat')]",closed,completed
2025-01-01T12:55:54Z,OWNER,opened,2025-01-01T12:55:52Z,[FE] PWA 설정,2765056197,"[Row(color='baffc9', default=False, description='프론트엔드', id=7950417303, name='🖼frontend', node_id='LA_kwDONSoEw88AAAAB2eG9lw', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%F0%9F%96%BCfrontend'), Row(color='464646', default=False, description='테스트 작업', id=7950417503, name='✅test', node_id='LA_kwDONSoEw88AAAAB2eG-Xw', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%E2%9C%85test')]",closed,completed
2025-01-01T12:55:56Z,OWNER,opened,2025-01-01T12:55:55Z,[BE] FastAPI 기본 구조 생성,2765056214,"[Row(color='ffdfba', default=False, description='백엔드', id=7950416460, name='💻backend', node_id='LA_kwDONSoEw88AAAAB2eG6TA', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%F0%9F%92%BBbackend'), Row(color='000000', default=False, description='개발 환경 셋팅', id=7950417398, name='⚙setting', node_id='LA_kwDONSoEw88AAAAB2eG99g', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%E2%9A%99setting')]",closed,completed
2025-01-01T12:56:00Z,OWNER,opened,2025-01-01T12:55:58Z,[BE] 음성 측정 함수,2765056241,"[Row(color='ffdfba', default=False, description='백엔드', id=7950416460, name='💻backend', node_id='LA_kwDONSoEw88AAAAB2eG6TA', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%F0%9F%92%BBbackend'), Row(color='00a800', default=False, description='기능 개발', id=7950417093, name='✨feat', node_id='LA_kwDONSoEw88AAAAB2eG8xQ', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%E2%9C%A8feat')]",closed,completed
2025-01-01T12:56:02Z,OWNER,opened,2025-01-01T12:56:01Z,[FE] 메인페이지,2765056262,"[Row(color='00a800', default=False, description='기능 개발', id=7950417093, name='✨feat', node_id='LA_kwDONSoEw88AAAAB2eG8xQ', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%E2%9C%A8feat'), Row(color='baffc9', default=False, description='프론트엔드', id=7950417303, name='🖼frontend', node_id='LA_kwDONSoEw88AAAAB2eG9lw', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%F0%9F%96%BCfrontend')]",closed,completed
2025-01-01T12:56:06Z,OWNER,opened,2025-01-01T12:56:04Z,"[BE/FE] FAST API 구축, 연결 테스트",2765056285,"[Row(color='ffdfba', default=False, description='백엔드', id=7950416460, name='💻backend', node_id='LA_kwDONSoEw88AAAAB2eG6TA', url='https://api.github.com/repos/BongSangKim/FixSpeech/labels/%F0%9F%92%BBbackend'), Row(color='00a800', default=False, description='기능 개발', id=7950417093, name='✨feat', node_id='LA_kwDONSoEw88AAAAB2eG8xQ', u

In [18]:
%%sparksql
SELECT created_at, type, payload.action, payload.issue.state, payload.issue.closed_at 
FROM df 
WHERE payload.issue.id = 2764760625 
ORDER BY created_at ASC;


created_at,type,action,state,closed_at
2025-01-01T01:05:05Z,IssuesEvent,closed,closed,2025-01-01T01:05:04Z
2025-01-01T01:05:06Z,IssuesEvent,opened,closed,2025-01-01T01:05:04Z


In [20]:
%%sparksql
SELECT 
    payload.issue.id AS issue_id, 
    MIN(created_at) AS first_opened, 
    MAX(payload.issue.closed_at) AS final_closed,
    TIMESTAMPDIFF(SECOND, MIN(created_at), MAX(payload.issue.closed_at)) AS resolution_time_seconds
FROM df
WHERE type = 'IssuesEvent' 
AND payload.issue.state = 'closed' 
AND payload.action = 'opened'
GROUP BY payload.issue.id
ORDER BY resolution_time_seconds DESC;


only showing top 20 row(s)


issue_id,first_opened,final_closed,resolution_time_seconds
2765057844,2025-01-01T12:59:14Z,2025-01-01T12:59:14Z,0
2765161495,2025-01-01T16:47:37Z,2025-01-01T16:47:37Z,0
2765058111,2025-01-01T12:59:44Z,2025-01-01T12:59:44Z,0
2764753864,2025-01-01T00:46:37Z,2025-01-01T00:46:37Z,0
2765058858,2025-01-01T13:01:02Z,2025-01-01T13:01:02Z,0
2764839439,2025-01-01T04:41:07Z,2025-01-01T04:41:07Z,0
2765059177,2025-01-01T13:01:38Z,2025-01-01T13:01:38Z,0
2764851081,2025-01-01T05:12:59Z,2025-01-01T05:12:59Z,0
2765078359,2025-01-01T13:40:56Z,2025-01-01T13:40:56Z,0
2764896188,2025-01-01T07:12:42Z,2025-01-01T07:12:42Z,0


In [24]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        created_at AS opened_at
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        created_at AS closed_at
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    TIMESTAMPDIFF(SECOND, o.opened_at, c.closed_at) AS time_to_close_seconds
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
ORDER BY time_to_close_seconds ASC;


only showing top 20 row(s)


issue_id,opened_at,closed_at,time_to_close_seconds
2764750585,2025-01-01T00:38:18Z,2025-01-01T23:58:06Z,83988
2761976744,2025-01-01T00:14:21Z,2025-01-01T23:04:30Z,82209
2764758445,2025-01-01T00:59:29Z,2025-01-01T23:35:21Z,81352
2764757468,2025-01-01T00:56:53Z,2025-01-01T23:15:13Z,80300
2764771640,2025-01-01T01:33:40Z,2025-01-01T23:48:17Z,80077
2764742246,2025-01-01T00:21:04Z,2025-01-01T22:33:16Z,79932
2764742305,2025-01-01T00:21:11Z,2025-01-01T22:33:23Z,79932
2764774114,2025-01-01T01:39:30Z,2025-01-01T23:51:03Z,79893
2764759266,2025-01-01T01:01:33Z,2025-01-01T22:58:47Z,79034
2764768065,2025-01-01T01:25:12Z,2025-01-01T23:21:31Z,78979


In [32]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MIN(created_at) AS opened_at  -- Pick first open event per issue
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')
    GROUP BY payload.issue.id
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MAX(created_at) AS closed_at  -- Pick last closed event per issue
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'
    GROUP BY payload.issue.id
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    TIMESTAMPDIFF(SECOND, o.opened_at, c.closed_at) AS time_to_close_seconds
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
WHERE c.closed_at >= o.opened_at  -- Ensure closed_at is after opened_at
ORDER BY time_to_close_seconds ASC;


only showing top 20 row(s)


issue_id,opened_at,closed_at,time_to_close_seconds
2764948588,2025-01-01T09:13:24Z,2025-01-01T09:13:24Z,0
2765232855,2025-01-01T19:50:05Z,2025-01-01T19:50:05Z,0
2764976428,2025-01-01T10:13:12Z,2025-01-01T10:13:12Z,0
2764753763,2025-01-01T00:46:18Z,2025-01-01T00:46:18Z,0
2765004989,2025-01-01T11:10:16Z,2025-01-01T11:10:16Z,0
2764812318,2025-01-01T03:24:30Z,2025-01-01T03:24:30Z,0
2765039832,2025-01-01T12:23:25Z,2025-01-01T12:23:25Z,0
2764836554,2025-01-01T04:33:26Z,2025-01-01T04:33:26Z,0
2765067215,2025-01-01T13:17:58Z,2025-01-01T13:17:58Z,0
2764844095,2025-01-01T04:53:58Z,2025-01-01T04:53:58Z,0


In [37]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MIN(created_at) AS opened_at,  -- Pick first open event per issue
        FIRST(payload.issue.author_association) AS author_association,  
        FIRST(payload.issue.title) AS title,
        FIRST(payload.issue.labels) AS labels,
        FIRST(payload.issue.state) AS state,
        FIRST(payload.issue.state_reason) AS state_reason,
        FIRST(payload.issue.body) AS body
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')
    GROUP BY payload.issue.id
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MAX(created_at) AS closed_at  -- Pick last closed event per issue
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'
    GROUP BY payload.issue.id
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    TIMESTAMPDIFF(SECOND, o.opened_at, c.closed_at) AS time_to_close_seconds,
    o.author_association,
    o.title,
    o.state,
    o.state_reason,
    o.body
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
WHERE c.closed_at >= o.opened_at AND o.state != 'open' -- Ensure closed_at is after opened_at
ORDER BY time_to_close_seconds DESC;


only showing top 20 row(s)


issue_id,opened_at,closed_at,time_to_close_seconds,author_association,title,state,state_reason,body
2765157628,2025-01-01T16:38:02Z,2025-01-01T16:38:33Z,31,NONE,issue-reopener-subdir pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1442 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572509435/attempts/1)
2765157618,2025-01-01T16:38:02Z,2025-01-01T16:38:32Z,30,NONE,issue-reopener-path pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1442 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572509435/attempts/1)
2765157679,2025-01-01T16:38:09Z,2025-01-01T16:38:38Z,29,NONE,issue-reopener-path pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1443 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572510530/attempts/1)
2765157609,2025-01-01T16:38:00Z,2025-01-01T16:38:27Z,27,NONE,issue-reopener-basic pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1442 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572509435/attempts/1)
2765157684,2025-01-01T16:38:10Z,2025-01-01T16:38:35Z,25,NONE,issue-reopener-vanityURLs pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1443 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572510530/attempts/1)
2765157704,2025-01-01T16:38:13Z,2025-01-01T16:38:35Z,22,NONE,issue-reopener-subdir pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1443 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572510530/attempts/1)
2765157757,2025-01-01T16:38:20Z,2025-01-01T16:38:36Z,16,NONE,issue-reopener-basic pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1444 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572511422/attempts/1)
2765161782,2025-01-01T16:48:15Z,2025-01-01T16:48:31Z,16,NONE,issue-reopener-vanityURLs pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1466 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572579639/attempts/1)
2765161505,2025-01-01T16:47:39Z,2025-01-01T16:47:54Z,15,NONE,issue-reopener-subdir pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1460 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572575247/attempts/1)
2765157823,2025-01-01T16:38:29Z,2025-01-01T16:38:44Z,15,NONE,issue-reopener-vanityURLs pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1444 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572511422/attempts/1)


In [14]:
%% sparksql
select * from df where type = 'IssuesEvent' and payload.action = 'opened' or payload.action = 'reopened' order by created_at desc limit 10;

UsageError: Cell magic `%%` not found.
